<a href="https://colab.research.google.com/github/juhumkwon/DeepLearning/blob/main/%EC%A0%95%EC%B1%85%EA%B8%B0%EB%B0%98_Reinforce%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import gym
import numpy as np
import tensorflow as tf

# 정책 신경망 (Policy Network)
class PolicyNet(tf.keras.Model):
    def __init__(self, action_size):
        super().__init__()
        self.d1 = tf.keras.layers.Dense(24, activation='relu')
        self.d2 = tf.keras.layers.Dense(24, activation='relu')
        self.out = tf.keras.layers.Dense(action_size, activation='softmax')

    def call(self, x):
        x = self.d1(x)
        x = self.d2(x)
        return self.out(x)  # 행동 확률 분포

# 에이전트
class REINFORCEAgent:
    def __init__(self, action_size, lr=0.01, gamma=0.99):
        self.action_size = action_size
        self.gamma = gamma
        self.model = PolicyNet(action_size)
        self.optimizer = tf.keras.optimizers.Adam(lr)
        self.states, self.actions, self.rewards = [], [], []

    def get_action(self, state):
        state = state.reshape([1, -1])
        probs = self.model(state)
        action = np.random.choice(self.action_size, p=probs.numpy()[0])
        return action

    def store(self, state, action, reward):
        self.states.append(state)
        self.actions.append(action)
        self.rewards.append(reward)

    def train(self):
        # 할인 리턴 계산
        returns = []
        G = 0
        for r in reversed(self.rewards):
            G = r + self.gamma * G
            returns.insert(0, G)
        returns = np.array(returns)
        returns = (returns - np.mean(returns)) / (np.std(returns) + 1e-8)

        # 학습
        with tf.GradientTape() as tape:
            loss = 0
            for state, action, Gt in zip(self.states, self.actions, returns):
                state = state.reshape([1, -1])
                probs = self.model(state)
                action_prob = probs[0, action]
                loss += -tf.math.log(action_prob) * Gt
        grads = tape.gradient(loss, self.model.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.model.trainable_variables))

        # 버퍼 초기화
        self.states, self.actions, self.rewards = [], [], []

# 환경 및 학습
env = gym.make("CartPole-v1")
agent = REINFORCEAgent(action_size=env.action_space.n)

EPISODES = 1000
for ep in range(EPISODES):
    state = env.reset()[0]
    done, score = False, 0
    while not done:
        action = agent.get_action(state)
        next_state, reward, done, _, _ = env.step(action)
        agent.store(state, action, reward)
        state = next_state
        score += reward

    agent.train()
    print(f"Episode {ep}, Score: {score}")

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/usr/local/lib/python3.11/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.11/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


AttributeError: module 'numpy' has no attribute 'bool8'

In [2]:
!pip install gymnasium stable-baselines3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.2/187.2 kB 697.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 34.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-c

In [ ]:
import gymnasium as gym
import numpy as np
import tensorflow as tf

# 정책 신경망 (Policy Network)
class PolicyNet(tf.keras.Model):
    def __init__(self, action_size):
        super().__init__()
        self.d1 = tf.keras.layers.Dense(24, activation='relu')
        self.d2 = tf.keras.layers.Dense(24, activation='relu')
        self.out = tf.keras.layers.Dense(action_size, activation='softmax')

    def call(self, x):
        x = self.d1(x)
        x = self.d2(x)
        return self.out(x)  # 행동 확률 분포

# 에이전트
class REINFORCEAgent:
    def __init__(self, action_size, lr=0.01, gamma=0.99):
        self.action_size = action_size
        self.gamma = gamma
        self.model = PolicyNet(action_size)
        self.optimizer = tf.keras.optimizers.Adam(lr)
        self.states, self.actions, self.rewards = [], [], []

    def get_action(self, state):
        state = state.reshape([1, -1])
        probs = self.model(state)
        action = np.random.choice(self.action_size, p=probs.numpy()[0])
        return action

    def store(self, state, action, reward):
        self.states.append(state)
        self.actions.append(action)
        self.rewards.append(reward)

    def train(self):
        # 할인 리턴 계산
        returns = []
        G = 0
        for r in reversed(self.rewards):
            G = r + self.gamma * G
            returns.insert(0, G)
        returns = np.array(returns)
        returns = (returns - np.mean(returns)) / (np.std(returns) + 1e-8)

        # 학습
        with tf.GradientTape() as tape:
            loss = 0
            for state, action, Gt in zip(self.states, self.actions, returns):
                state = state.reshape([1, -1])
                probs = self.model(state)
                action_prob = probs[0, action]
                loss += -tf.math.log(action_prob) * Gt
        grads = tape.gradient(loss, self.model.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.model.trainable_variables))

        # 버퍼 초기화
        self.states, self.actions, self.rewards = [], [], []

# 환경 및 학습
env = gym.make("CartPole-v1")
agent = REINFORCEAgent(action_size=env.action_space.n)

EPISODES = 1000
for ep in range(EPISODES):
    state, info = env.reset()  # Gymnasium reset returns state and info
    done, score = False, 0
    while not done:
        action = agent.get_action(state)
        next_state, reward, terminated, truncated, info = env.step(action) # Gymnasium step returns terminated and truncated
        done = terminated or truncated # Combine terminated and truncated for done
        agent.store(state, action, reward)
        state = next_state
        score += reward

    agent.train()
    print(f"Episode {ep}, Score: {score}")

env.close()

Episode 0, Score: 16.0
Episode 1, Score: 33.0
Episode 2, Score: 21.0
Episode 3, Score: 24.0
Episode 4, Score: 20.0
Episode 5, Score: 33.0
Episode 6, Score: 18.0
Episode 7, Score: 17.0
Episode 8, Score: 17.0
Episode 9, Score: 46.0
Episode 10, Score: 19.0
Episode 11, Score: 15.0
Episode 12, Score: 23.0
Episode 13, Score: 14.0
Episode 14, Score: 15.0
Episode 15, Score: 28.0
Episode 16, Score: 47.0
Episode 17, Score: 28.0
Episode 18, Score: 38.0
Episode 19, Score: 24.0
Episode 20, Score: 20.0
Episode 21, Score: 14.0
Episode 22, Score: 36.0
Episode 23, Score: 26.0
Episode 24, Score: 55.0
Episode 25, Score: 52.0
Episode 26, Score: 20.0
Episode 27, Score: 11.0
Episode 28, Score: 37.0
Episode 29, Score: 56.0
Episode 30, Score: 58.0
Episode 31, Score: 34.0
Episode 32, Score: 13.0
Episode 33, Score: 74.0
Episode 34, Score: 105.0
Episode 35, Score: 29.0
Episode 36, Score: 38.0
Episode 37, Score: 55.0
Episode 38, Score: 20.0
Episode 39, Score: 106.0
Episode 40, Score: 36.0
Episode 41, Score: 19.0
